# deletes tweets from a file
Remember to download your twitter archive first, and use the tweet sorter if you want to be selective about what you delete.

### imports and setup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time
from tqdm import tqdm

In [ ]:
service = webdriver.ChromeService(executable_path = './chromedriver')
driver = webdriver.Chrome(service=service)

## settings

In [ ]:
tweets_to_delete = 'tweets_delete.js'
your_twitter_handle = '' #put your twitter handle here, without the @

## login
Run the code below and a browser window with twitter will pop up. Log into your account there before continuing to run the code here.

In [ ]:
url = 'https://twitter.com/'
driver.get(url)

## load data

In [ ]:
with open(tweets_to_delete, 'rt') as delfile:
    data = json.loads(delfile.read()[26:])

In [ ]:
len(data)

## delete tweets

In [ ]:
for t in tqdm(data):
    skip=False
    tid = t['tweet']['id']
    url = 'https://twitter.com/'+your_twitter_handle+'/status/'+tid
    driver.get(url)
    time.sleep(3)

    #grab all the divs. Twitter randomizes their class names, so we do it the hard way
    matches = driver.find_elements(By.TAG_NAME, 'div')

    for m in matches:
        try:
            #skip 404 or otherwise unavailable tweets.
            if 'error-detail' == m.get_attribute('data-testid'):
                skip=True
                break
        except:
            pass
    if skip:
        continue
    
    for m in matches:
        ar = ''
        try:
            ar = m.aria_role 
        except:
            pass
            
        #enumerate the buttons on the page
        if ar == 'button':
            al = m.get_attribute('aria-label')
            ah = m.get_attribute('aria-haspopup')
            
            #click all the the menu buttons
            if (al == 'More') and (ah=='menu'):
                try:
                    m.click()
                except:
                    continue
                try:
                    #if the menu lets us delete, delete it. 
                    d = driver.find_element(By.XPATH, "//span[.='Delete']")
                    d.click()
                    #now repeat to confirm
                    d = driver.find_element(By.XPATH, "//span[.='Delete']")
                    d.click()
                except:
                    #if there's no delete in this menu, click out of it before we go to the next menu
                    try:
                        driver.find_element(By.ID, 'react-root').click()
                    except:
                        #even this fails sometimes. skip for now, clean up later, maybe manually
                        pass